In [3]:
import pandas as pd
import os
from PIL import Image, ImageDraw, ImageFont
from rembg import remove
from io import BytesIO

# Carrega o CSV com os dados
df = pd.read_csv('./datasets/government_positions.csv', index_col=0)

# Pastas
background_path = './background/background.png'
palestrante_dir = './speakers/'
output_dir = './output/'
os.makedirs(output_dir, exist_ok=True)

# Carrega o background
background = Image.open(background_path).convert("RGBA")

# Loop nos palestrantes
for nome_arquivo in os.listdir(palestrante_dir):
    if nome_arquivo.endswith(('.jpg', '.jpeg', '.png')):
        caminho_foto = os.path.join(palestrante_dir, nome_arquivo)
        print(f"Processando: {nome_arquivo}")

        try:
            # Remove fundo com rembg
            with open(caminho_foto, "rb") as f:
                img_data = f.read()
                pessoa_sem_fundo = remove(img_data)
                pessoa_img = Image.open(BytesIO(pessoa_sem_fundo)).convert("RGBA")

                # Cropar a imagem com base no conteúdo visível (onde alpha > 0)
                bbox = pessoa_img.getbbox()  # retorna (left, upper, right, lower)
                if bbox:
                    pessoa_img = pessoa_img.crop(bbox)

            # Redimensiona mantendo proporção para 85% da altura do background
            bg_width, bg_height = background.size
            target_height = int(bg_height * 0.85)

            # Mantém proporção da imagem da pessoa
            pessoa_width, pessoa_height = pessoa_img.size
            aspect_ratio = pessoa_width / pessoa_height
            new_width = int(target_height * aspect_ratio)

            pessoa_redimensionada = pessoa_img.resize((new_width, target_height), Image.LANCZOS)

            # Posiciona no canto direito da imagem (horizontalmente alinhado à direita)
            x_pos = bg_width - new_width - 120  # margem direita
            y_pos = bg_height - target_height # margem inferior

            flyer_final = background.copy()
            flyer_final.paste(pessoa_redimensionada, (x_pos, y_pos), pessoa_redimensionada)

           # Criar faixa verde com textos
            draw = ImageDraw.Draw(flyer_final)

            # Fonte Times New Roman
            fonte_nome = ImageFont.truetype("times.ttf", size=34)
            fonte_ocupacao = ImageFont.truetype("times.ttf", size=28)

            # Dimensões da faixa
            largura_faixa = int(bg_width * 0.6)
            altura_faixa = 120
            x_faixa = bg_width - largura_faixa  # Alinhado à direita
            y_faixa = bg_height - altura_faixa - 70

            # Simular faixa com cantos arredondados só à esquerda
            radius = 10
            faixa_color = "#006942"

            # Retângulo principal
            draw.rectangle(
                [x_faixa + radius, y_faixa, x_faixa + largura_faixa, y_faixa + altura_faixa],
                fill=faixa_color
            )

            # Retângulo esquerdo (onde fica o arredondamento)
            draw.rectangle(
                [x_faixa, y_faixa + radius, x_faixa + radius, y_faixa + altura_faixa - radius],
                fill=faixa_color
            )

            # Canto superior esquerdo
            draw.pieslice(
                [x_faixa, y_faixa, x_faixa + 2 * radius, y_faixa + 2 * radius],
                180, 270, fill=faixa_color
            )

            # Canto inferior esquerdo
            draw.pieslice(
                [x_faixa, y_faixa + altura_faixa - 2 * radius, x_faixa + 2 * radius, y_faixa + altura_faixa],
                90, 180, fill=faixa_color
            )

            # Extrair nome do arquivo sem extensão
            nome_base = os.path.splitext(nome_arquivo)[0]

            # Acessa os dados no CSV
            try:
                speaker_name = df.loc[nome_base, 'name']
                position = df.loc[nome_base, 'position']
                country = df.loc[nome_base, 'country']
                texto_exibido = f"{speaker_name} - {position}, {country}"
            except KeyError:
                print(f"[ERRO] {nome_base} não encontrado no CSV.")
                texto_exibido = "Speaker Name - Position, Country"

            # Fonte Times New Roman maior
            fonte_texto = ImageFont.truetype("times.ttf", size=44)

            # Medidas para centralizar verticalmente
            text_height = fonte_texto.getbbox(texto_exibido)[3]
            padding_left = 50
            text_y = y_faixa + (altura_faixa - text_height) // 2

            # Desenhar o texto na faixa
            draw.text((x_faixa + padding_left, text_y), texto_exibido, font=fonte_texto, fill="white")

            # Faixa inferior com informação
            faixa_info_altura = 50
            y_faixa_info = bg_height - faixa_info_altura

            # Desenhar a faixa branca inferior
            draw.rectangle(
                [0, y_faixa_info, bg_width, bg_height],
                fill="white"
            )

            # Texto da faixa
            texto_info = "For more information: www.islandinnovation.co/GSIS2025"
            fonte_info = ImageFont.truetype("times.ttf", size=24)
            text_color = "#004225"
            padding_text = 108

            # Centralizar verticalmente o texto
            text_height = fonte_info.getbbox(texto_info)[3]
            text_y_info = y_faixa_info + (faixa_info_altura - text_height) // 2

            draw.text((padding_text, text_y_info), texto_info, font=fonte_info, fill=text_color)

            # Salva resultado em ./Results
            nome_base = os.path.splitext(nome_arquivo)[0]
            flyer_final.save(os.path.join(output_dir, f"placard_{nome_base}.png"))

        except Exception as e:
            print(f"Erro ao processar {nome_arquivo}: {e}")

Processando: ahmed-hassan.jpg
Processando: aisha-al-fulan.jpg
Processando: amelie-moreau.jpg
Processando: carlos-hernandez.jpg
Processando: emily-johnson.jpg
Processando: hiroshi-tanaka.jpg
Processando: ivan-petrov.jpg
Processando: jean-dupont.jpg
Processando: john-smith.jpg
Processando: kwame-nkrumah.jpg
Processando: liam-oconnor.jpg
Processando: olivia-brown.jpg
Processando: paulo-silva.jpg
Processando: shrek.png
Processando: sofia-garcia.jpg


In [5]:
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from rembg import remove
from io import BytesIO

# --- CONFIGURAÇÕES ---
background_path = './background/background.png'
palestrante_dir = './speakers/'
output_dir = './output/'
csv_path = './datasets/government_positions.csv'

fonte_principal = ImageFont.truetype("times.ttf", size=44)
fonte_nome = ImageFont.truetype("times.ttf", size=34)
fonte_info = ImageFont.truetype("times.ttf", size=24)
text_color = "white"
green_color = "#006942"
footer_color = "white"
footer_text_color = "#004225"

# --- PREPARAÇÃO ---
os.makedirs(output_dir, exist_ok=True)
df = pd.read_csv(csv_path, index_col=0)
background = Image.open(background_path).convert("RGBA")
bg_width, bg_height = background.size


def desenhar_faixa_verde(draw, x_faixa, y_faixa, largura, altura, radius):
    draw.rectangle([x_faixa + radius, y_faixa, x_faixa + largura, y_faixa + altura], fill=green_color)
    draw.rectangle([x_faixa, y_faixa + radius, x_faixa + radius, y_faixa + altura - radius], fill=green_color)
    draw.pieslice([x_faixa, y_faixa, x_faixa + 2 * radius, y_faixa + 2 * radius], 180, 270, fill=green_color)
    draw.pieslice([x_faixa, y_faixa + altura - 2 * radius, x_faixa + 2 * radius, y_faixa + altura], 90, 180, fill=green_color)


def remover_fundo_imagem(caminho_foto):
    with open(caminho_foto, "rb") as f:
        img_data = f.read()
        pessoa_sem_fundo = remove(img_data)
        img = Image.open(BytesIO(pessoa_sem_fundo)).convert("RGBA")
        return img.crop(img.getbbox()) if img.getbbox() else img


# --- LOOP PRINCIPAL ---
for nome_arquivo in os.listdir(palestrante_dir):
    if not nome_arquivo.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    nome_base = os.path.splitext(nome_arquivo)[0]
    caminho_foto = os.path.join(palestrante_dir, nome_arquivo)
    print(f"Processando: {nome_arquivo}")

    try:
        pessoa_img = remover_fundo_imagem(caminho_foto)

        # Redimensiona imagem da pessoa
        target_height = int(bg_height * 0.85)
        aspect_ratio = pessoa_img.width / pessoa_img.height
        new_width = int(target_height * aspect_ratio)
        pessoa_redimensionada = pessoa_img.resize((new_width, target_height), Image.LANCZOS)

        # Posição da imagem da pessoa
        x_pos = bg_width - new_width - 120
        y_pos = bg_height - target_height

        # Monta a imagem final
        flyer_final = background.copy()
        flyer_final.paste(pessoa_redimensionada, (x_pos, y_pos), pessoa_redimensionada)

        draw = ImageDraw.Draw(flyer_final)

        # Faixa inferior (branca)
        footer_height = 50
        y_footer = bg_height - footer_height
        draw.rectangle([0, y_footer, bg_width, bg_height], fill=footer_color)

        texto_info = "For more information: www.islandinnovation.co/GSIS2025"
        text_height = fonte_info.getbbox(texto_info)[3]
        draw.text((108, y_footer + (footer_height - text_height) // 2), texto_info, font=fonte_info, fill=footer_text_color)

        # Faixa verde
        largura_faixa = int(bg_width * 0.6)
        altura_faixa = 120
        x_faixa = bg_width - largura_faixa
        y_faixa = bg_height - altura_faixa - 70
        desenhar_faixa_verde(draw, x_faixa, y_faixa, largura_faixa, altura_faixa, radius=10)

        # Texto principal
        try:
            dados = df.loc[nome_base]
            texto_exibido = f"{dados['name']} - {dados['position']}, {dados['country']}"
        except KeyError:
            texto_exibido = "Speaker Name - Position, Country"
            print(f"[ERRO] '{nome_base}' não encontrado no CSV.")

        text_height = fonte_principal.getbbox(texto_exibido)[3]
        draw.text((x_faixa + 50, y_faixa + (altura_faixa - text_height) // 2), texto_exibido, font=fonte_principal, fill=text_color)

        # --- salvar
        output_path = os.path.join(output_dir, f"placard_{nome_base}.png")
        flyer_final.save(output_path)

    except Exception as e:
        print(f"[ERRO] Falha ao processar '{nome_arquivo}': {e}")


Processando: ahmed-hassan.jpg
Processando: aisha-al-fulan.jpg
Processando: amelie-moreau.jpg
Processando: batman.png
Processando: carlos-hernandez.jpg
Processando: emily-johnson.jpg
Processando: hiroshi-tanaka.jpg
Processando: ivan-petrov.jpg
Processando: jack-sparrow.png
Processando: jean-dupont.jpg
Processando: john-smith.jpg
Processando: kwame-nkrumah.jpg
Processando: liam-oconnor.jpg
Processando: olivia-brown.jpg
Processando: paulo-silva.jpg
Processando: shrek.png
Processando: sofia-garcia.jpg
